In [98]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/blog-authorship-corpus/blogtext.csv


#### Read the csv using pandas

In [99]:
import pandas as pd

df = pd.read_csv("/kaggle/input/blog-authorship-corpus/blogtext.csv")

#### Delete data_csv variable for memory

#### Get the names of the columns

In [100]:
df.columns

Index(['id', 'gender', 'age', 'topic', 'sign', 'date', 'text'], dtype='object')

#### Have a look at some column values

In [101]:
df.head(5)

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


#### Check if there is any null value, and get the total count.

In [102]:
df.isnull().sum()

id        0
gender    0
age       0
topic     0
sign      0
date      0
text      0
dtype: int64

### Cut the data (skip this step in final run)
Make your data short during development. So that overall process takes less time to execute and you are able to rectify all the errors fast, and check if your code is running smooth.
When evrything is sorted at last, load the entire data and run your code on that and skip this step.

In [103]:
df = df.head(3000)

## Preprocess text
Preprocess values of text column

- Remove unwanted characters
- Convert text to lowercase
- Remove unwanted spaces
- Remove stopwords

In [104]:
# Select only alphabets
import re
df.text = df.text.apply(lambda x: re.sub('[^A-Za-z]+', ' ', x))

# Convert text to lowercase
df.text = df.text.apply(lambda x: x.lower())

# Strip unwanted spaces
df.text = df.text.apply(lambda x: x.strip())

# Remove stopwords
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
df.text = df.text.apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]))

Verify the preprocessing steps by looking over some values

In [105]:
df.text[6]

'somehow coca cola way summing things well early flagship jingle like buy world coke tune like teach world sing pretty much summed post woodstock era well add much sales catchy tune korea coke theme urllink stop thinking feel pretty much sums lot korea koreans look relaxed couple stopped thinking started feeling course high regard education math logic deep think many koreans really like work emotion anything else westerners seem sublimate moreso least display different way maybe scratch westerners koreans probably pretty similar context different anyways think losing korea repeat stop thinking feel stop thinking feel stop thinking feel everything alright'

### Merge the label columns

Merge all the label columns together, so that we have all the tags together for a particular sentence

In [106]:
df['labels'] = df.apply(lambda row: [row['gender'], str(row['age']), row['topic'], row['sign']], axis=1)

### Select only required columns from your dataframe

In [107]:
df = df[['text','labels']]

### Print final dataframe

In [108]:
df.head()

,text,labels
0,info found pages mb pdf files wait untill team...,"[male, 15, Student, Leo]"
1,team members drewes van der laag urllink mail ...,"[male, 15, Student, Leo]"
2,het kader van kernfusie op aarde maak je eigen...,"[male, 15, Student, Leo]"
3,testing testing,"[male, 15, Student, Leo]"
4,thanks yahoo toolbar capture urls popups means...,"[male, 33, InvestmentBanking, Aquarius]"


## Create training and testing data

In [109]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.text.values, df.labels.values, test_size=0.20, random_state=42)

## Vectorize the data

### Create Bag of Words
- Use CountVectorizer
- Transform the training and testing data

In [110]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1, 2))
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

#### Have a look at some feature names

In [111]:
vectorizer.get_feature_names()[:5]

['aa', 'aa compared', 'aa nice', 'aaa', 'aaa take']

#### View term-document matrix

In [112]:
X_train_bow.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [113]:
X_test_bow.shape

(600, 180703)

### Create a dictionary to get label counts

In [114]:
label_counts = dict()

for labels in df.labels.values:
    for label in labels:
        if label in label_counts:
            label_counts[label] += 1
        else:
            label_counts[label] = 1

#### Print the dictionary

In [115]:
label_counts

{'male': 2272,
 '15': 299,
 'Student': 403,
 'Leo': 55,
 '33': 94,
 'InvestmentBanking': 70,
 'Aquarius': 286,
 'female': 728,
 '14': 74,
 'indUnk': 452,
 'Aries': 1699,
 '25': 110,
 'Capricorn': 77,
 '17': 147,
 'Gemini': 21,
 '23': 93,
 'Non-Profit': 46,
 'Cancer': 76,
 'Banking': 16,
 '37': 19,
 'Sagittarius': 113,
 '26': 43,
 '24': 334,
 'Scorpio': 243,
 '27': 86,
 'Education': 118,
 '45': 14,
 'Engineering': 119,
 'Libra': 313,
 'Science': 33,
 '34': 6,
 '41': 14,
 'Communications-Media': 14,
 'BusinessServices': 21,
 'Sports-Recreation': 75,
 'Virgo': 39,
 'Taurus': 76,
 'Arts': 2,
 'Pisces': 2,
 '44': 3,
 '16': 25,
 'Internet': 20,
 'Museums-Libraries': 2,
 'Accounting': 2,
 '39': 32,
 '35': 1607,
 'Technology': 1607}

In [116]:
y_train.shape

(2400,)

## Multi label binarizer

Load a multilabel binarizer and fit it on the labels.

In [117]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(classes=sorted(label_counts.keys()))
y_train = mlb.fit_transform(y_train)
y_test = mlb.transform(y_test)

In [118]:
y_train

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 1],
       ...,
       [0, 1, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

## Classifier

Use a linear classifier of your choice, wrap it up in OneVsRestClassifier to train it on every label.

In [119]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(solver='lbfgs')
clf = OneVsRestClassifier(clf)

### Fit the classifier

In [ ]:
clf.fit(X_train_bow, y_train)

## Make predictions
- Get predicted labels and scores

In [ ]:
predicted_labels = clf.predict(X_test_bow)
predicted_scores = clf.decision_function(X_test_bow)

In [ ]:
predicted_scores.shape

In [ ]:
predicted_labels.shape

### Get inverse transform for predicted labels and test labels

In [ ]:
pred_inversed = mlb.inverse_transform(predicted_labels)
y_test_inversed = mlb.inverse_transform(y_test)

### Print some samples

In [ ]:
for i in range(5):
    print('Title:\t{}\nTrue labels:\t{}\nPredicted labels:\t{}\n\n'.format(
        X_test[i],
        ','.join(y_test_inversed[i]),
        ','.join(pred_inversed[i])
    ))

## Calculate accuracy
- Accuracy
- F1-score
- Precision
- Recall

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score

def print_evaluation_scores(y_val, predicted):
    print('Accuracy score: ', accuracy_score(y_val, predicted))
    print('F1 score: ', f1_score(y_val, predicted, average='weighted'))
    print('Average precision score: ', average_precision_score(y_val, predicted, average='weighted'))
    print('Average recall score: ', recall_score(y_val, predicted, average='weighted'))

In [ ]:
print('Bag-of-words')
print_evaluation_scores(y_test, predicted_labels)